In [1]:
import numpy as np
import matplotlib.pyplot as plt
import autokeras as ak
import tensorflow as tf
import os
tf.keras.backend.set_floatx('float64')

In [2]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5000)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPU


In [3]:
data = np.load('/home/ML4NO/Data/n1000000_0910_all_flat.npz')

In [4]:
data_all = np.column_stack([data['ve_dune'], data['vu_dune'], data['vebar_dune'], data['vubar_dune'], data['ve_t2hk'], data['vu_t2hk'], data['vebar_t2hk'], data['vubar_t2hk']])
target = data['delta']

x_train = data_all[:10000]
y_train = target[:10000]

x_train2 = data_all[10000:900000]
y_train2 = target[10000:900000]

x_test = data_all[900000:]
y_test = target[900000:]

In [5]:
clf = ak.StructuredDataRegressor(overwrite=True, max_trials=50)
clf.fit(x_train, y_train,
           validation_split = 0.1,
           batch_size=64,
           epochs=20,
           verbose=1,
           shuffle = True
       )

Trial 50 Complete [00h 01m 00s]
val_loss: 518.258056640625

Best val_loss So Far: 162.29811096191406
Total elapsed time: 00h 48m 25s
INFO:tensorflow:Oracle triggered exit
Epoch 1/20
157/157 [==============================] - 5s 16ms/step - loss: 34313.5732 - mean_squared_error: 34313.5732
Epoch 2/20
157/157 [==============================] - 3s 17ms/step - loss: 4332.8401 - mean_squared_error: 4332.8401
Epoch 3/20
157/157 [==============================] - 3s 17ms/step - loss: 1825.4702 - mean_squared_error: 1825.4702
Epoch 4/20
157/157 [==============================] - 3s 16ms/step - loss: 1081.1547 - mean_squared_error: 1081.1547
Epoch 5/20
157/157 [==============================] - 2s 16ms/step - loss: 855.6337 - mean_squared_error: 855.6337
Epoch 6/20
157/157 [==============================] - 3s 17ms/step - loss: 757.4258 - mean_squared_error: 757.4258
Epoch 7/20
157/157 [==============================] - 3s 16ms/step - loss: 655.8836 - mean_squared_error: 655.8836
Epoch 8/20
157

In [6]:
model = clf.export_model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 304)]             0         
_________________________________________________________________
multi_category_encoding (Mul (None, 304)               0         
_________________________________________________________________
normalization (Normalization (None, 304)               609       
_________________________________________________________________
dense (Dense)                (None, 256)               78080     
_________________________________________________________________
re_lu (ReLU)                 (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
re_lu_1 (ReLU)               (None, 256)               0     

In [7]:
model.fit(x_train2, y_train2,
           validation_split = 0.1,
           batch_size=64,
           epochs=20,
           verbose=1,
           shuffle = True
         )

Epoch 1/20
12516/12516 [==============================] - 223s 18ms/step - loss: 241.8868 - mean_squared_error: 241.8868 - val_loss: 107.0816 - val_mean_squared_error: 107.0816
Epoch 2/20
12516/12516 [==============================] - 220s 18ms/step - loss: 195.9847 - mean_squared_error: 195.9847 - val_loss: 107.0816 - val_mean_squared_error: 107.0816
Epoch 3/20
12516/12516 [==============================] - 219s 18ms/step - loss: 196.2262 - mean_squared_error: 196.2262 - val_loss: 107.0816 - val_mean_squared_error: 107.0816
Epoch 4/20
12516/12516 [==============================] - 220s 18ms/step - loss: 196.4744 - mean_squared_error: 196.4744 - val_loss: 107.0816 - val_mean_squared_error: 107.0816
Epoch 5/20
12516/12516 [==============================] - 219s 18ms/step - loss: 195.9557 - mean_squared_error: 195.9557 - val_loss: 107.0816 - val_mean_squared_error: 107.0816
Epoch 6/20
12516/12516 [==============================] - 220s 18ms/step - loss: 196.4931 - mean_squared_error: 196

In [8]:
index = 1
while os.path.isfile('/home/ML4NO/ML/Regression/models/0910_delta_{}.h5'.format(index)):
    index += 1
model.save('/home/ML4NO/ML/Regression/models/0910_delta_{}.h5'.format(index))

In [9]:
scale_steps = np.logspace(-3, 0, 30)
before_train_loss = []
after_train_loss = []

for scale in scale_steps:
    x_train2_gen = np.random.normal(x_train2, np.sqrt(x_train2)*scale)
    x_test_gen = np.random.normal(x_test, np.sqrt(x_test)*scale)

    before_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

    model.fit(x_train2_gen, y_train2,
               validation_split = 0.1,
               batch_size=64,
               epochs=5,
               verbose=1,
               shuffle = True
             )

    after_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

3125/3125 [==============================] - 36s 11ms/step - loss: 121.7485 - mean_squared_error: 121.7485
Epoch 1/5
12516/12516 [==============================] - 221s 18ms/step - loss: 196.2951 - mean_squared_error: 196.2951 - val_loss: 107.3397 - val_mean_squared_error: 107.3397
Epoch 2/5
12516/12516 [==============================] - 223s 18ms/step - loss: 196.4579 - mean_squared_error: 196.4579 - val_loss: 107.3397 - val_mean_squared_error: 107.3397
Epoch 3/5
12516/12516 [==============================] - 220s 18ms/step - loss: 196.4359 - mean_squared_error: 196.4359 - val_loss: 107.3397 - val_mean_squared_error: 107.3397
Epoch 4/5
12516/12516 [==============================] - 220s 18ms/step - loss: 196.0954 - mean_squared_error: 196.0954 - val_loss: 107.3397 - val_mean_squared_error: 107.3397
Epoch 5/5
3125/3125 [==============================] - 38s 12ms/step - loss: 121.9529 - mean_squared_error: 121.9529
Epoch 1/5
12516/12516 [==============================] - 229s 18ms/step 

In [10]:
model_index = index
index = 1
path = '/home/ML4NO/ML/Regression/models_furthurTrain/0910_delta_{}_{}.h5'
while os.path.isfile(path.format(model_index, index)):
    index += 1
model.save(path.format(model_index, index))
outfile = {'scale_steps': scale_steps,
           'before_train_loss': before_train_loss,
           'after_train_loss' :after_train_loss}
np.save(file = '/home/ML4NO/ML/Regression/models_furthurTrain/0910_delta_{}_{}_result.npy'.format(model_index, index),
        arr = outfile)

In [11]:
x_test2_gen = np.random.poisson(x_test)

for i in range(10):
    x_train2_gen = np.random.poisson(x_train2)
    
    model.fit(x_train2_gen, y_train2,
              validation_split=0.1,
               batch_size=64,
               epochs=1,
               verbose=1,
               shuffle = True
             )
model.evaluate(x_test2_gen, y_test)

3125/3125 [==============================] - 43s 14ms/step - loss: 72223.6641 - mean_squared_error: 72223.6641


[72223.6640625, 72223.6640625]

In [12]:
furthur_index = index
index = 1
path = '/home/ML4NO/ML/Regression/models_PoissonTrain/0910_delta_{}_{}_{}.h5'
while os.path.isfile(path.format(model_index, furthur_index, index)):
    index += 1
model.save(path.format(model_index, furthur_index, index))